In [1]:
from deepface import DeepFace
from retinaface import RetinaFace
import cv2
import time
import mediapipe as mp
import winsound
import numpy as np
import copy

((array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ...

In [2]:
def extract_persons(image):

    def get_output_layers(net):
        
        layer_names = net.getLayerNames()
        try:
            output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
        except:
            output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

        return output_layers


    def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

        label = str(classes[class_id])

        color = COLORS[class_id]

        cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

        cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        

    Width = image.shape[1]
    Height = image.shape[0]
    scale = 0.00392

    classes = None

    with open("yolov3.txt", 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

    net = cv2.dnn.readNet("yolov3-tiny.weights", "yolo_tiny.cfg")

    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

    net.setInput(blob)

    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4
    cropped_image_arr = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])


    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    image_drawn = copy.copy(image)
    for i in indices:
        try:
            box = boxes[i]
        except:
            i = i[0]
            box = boxes[i]
        
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        draw_prediction(image_drawn, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
        try:
            cropped_image = image[round(y):round(y+h),round(x):round(x+w)] 
            cropped_image_arr.append(cropped_image) 
        except:
            pass
       

    return cropped_image_arr,image_drawn



In [9]:
import cv2

img = cv2.imread("indhu.jpg")
img.shape[0] > 0

True

In [3]:
#vid = cv2.VideoCapture(0)
frame_time = time.time()
frame_time_for_pose = time.time()
no_of_frames = 0
face_count = 0
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5)
pose_array = []
pose_array_temp = []
criminal_match = {}

#vid = cv2.VideoCapture('people-walking.mp4')
vid = cv2.VideoCapture(0)

while True:
    ret, frame = vid.read()
    persons, image_by_yolo = extract_persons(frame)
    cv2.imshow("person detection1", image_by_yolo)        


    print(persons)
    if 1:#try:
        pose_results = pose.process(frame)
        try:
            facial_data = DeepFace.extract_faces(frame)
        except:
            pass
        #facial_data = RetinaFace.detect_faces(frame)
        #mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        for face in facial_data:
            x = face['facial_area']['x']
            y = face['facial_area']['y']
            h = face['facial_area']['x'] + face['facial_area']['h']
            w = face['facial_area']['y'] + face['facial_area']['w']

            frame = cv2.rectangle(frame,(x,y),(h,w),(255, 0, 0),thickness = 2)
            cropped_image = frame[y:w,x:h] 
            #cv2.imshow("output",cropped_image) 
            cv2.imwrite("D:\\mini project2\\vid to frame\\frame%d.jpg" % face_count, cropped_image)
            face_count+=1
            try:
                criminal_match = DeepFace.verify("frame35.jpg",cropped_image,model_name='Facenet',enforce_detection=False)
            except:
                criminal_match['verified'] = False
            if criminal_match['distance'] <= 0.1 and criminal_match['verified']:
                print("criminal match = ",criminal_match)
                cropped_image = cv2.resize(cropped_image, (300,300), interpolation = cv2.INTER_AREA)

                cv2.imshow("criminal",cropped_image)
                cv2.imwrite("D:\\mini project2\\criminals\\frame%d.jpg" % face_count, cropped_image)
            else:
                print("criminal match = ",criminal_match)

            
        for person in persons:
            #cv2.imshow("person detection", person)        

            pose_results = pose.process(person)
            mp_drawing.draw_landmarks(person, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            cv2.imshow("person detection", person)             


        temp_time_pose = time.time() - frame_time_for_pose
        pose_array_temp.append(pose_results.pose_landmarks)

        if temp_time_pose >= 1.5:
            frame_time_for_pose = 0
            pose_array.append(pose_array_temp)

        winsound.Beep(2500,500)
        
    else:#except:
        print("can't find faces")
        

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    temp_time = time.time() - frame_time
    no_of_frames += 1
    if temp_time >= 1:
        print("frames processed : ",no_of_frames)
        no_of_frames = 0
        frame_time = time.time()

vid.release()
cv2.destroyAllWindows()

[]
criminal match =  {'verified': True, 'distance': 0.315845964123292, 'threshold': 0.4, 'model': 'Facenet', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 3, 'y': 3, 'w': 53, 'h': 53}, 'img2': {'x': 0, 'y': 0, 'w': 214, 'h': 214}}, 'time': 6.51}
frames processed :  1
[]
criminal match =  {'verified': True, 'distance': 0.27843893887687376, 'threshold': 0.4, 'model': 'Facenet', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 3, 'y': 3, 'w': 53, 'h': 53}, 'img2': {'x': 19, 'y': 15, 'w': 181, 'h': 181}}, 'time': 0.17}
[]
criminal match =  {'verified': True, 'distance': 0.2557951606450276, 'threshold': 0.4, 'model': 'Facenet', 'detector_backend': 'opencv', 'similarity_metric': 'cosine', 'facial_areas': {'img1': {'x': 3, 'y': 3, 'w': 53, 'h': 53}, 'img2': {'x': 14, 'y': 14, 'w': 181, 'h': 181}}, 'time': 0.16}
frames processed :  2
[]
criminal match =  {'verified': True, 'distance': 0.2426139937695163, '

In [7]:
vid = cv2.VideoCapture('people-walking.mp4')
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5)
while True:
    ret, frame = vid.read()
    persons, image_by_yolo = extract_persons(frame)
    cv2.imshow("person detection1", image_by_yolo)   

    for person in persons:
        cv2.imshow("person detection", person)        

        pose_results = pose.process(person)
        mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
vid.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [ ]:
DeepFace.verify("frame35.jpg","frame136.jpg",enforce_detection=False,model_name='Facenet')

In [ ]:
frame = cv2.imread("face_crowd1.jpg")

frame = cv2.rectangle(frame,(1000,500),(300,300),(0, 0, 0),thickness = 2)
cv2.imshow("output",frame)
cv2.waitKey(0)


In [ ]:
#vid = cv2.VideoCapture(0)
frame_time = time.time()
frame_time_for_pose = time.time()
no_of_frames = 0
face_count = 0
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5)
pose_array = []
pose_array_temp = []


vid = cv2.VideoCapture('people-walking.mp4')

#while True:
for i in range(1):

    #ret, frame = vid.read()
    frame = cv2.imread("face.jpg")
    try:
        pose_results = pose.process(frame)
        facial_data = DeepFace.extract_faces(frame)
        #facial_data = RetinaFace.detect_faces(frame)
        mp_drawing.draw_landmarks(frame, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        for face in facial_data:
            x = face['facial_area']['x']
            y = face['facial_area']['y']
            h = face['facial_area']['x'] + face['facial_area']['h']
            w = face['facial_area']['y'] + face['facial_area']['w']

            frame = cv2.rectangle(frame,(x,y),(h,w),(255, 0, 0),thickness = 2)
            cropped_image = frame[y:w,x:h]  
            cv2.imwrite("D:\\mini project2\\vid to frame\\frame%d.jpg" % face_count, cropped_image)
            face_count+=1

        temp_time_pose = time.time() - frame_time_for_pose
        pose_array_temp.append(pose_results.pose_landmarks)

        if temp_time_pose >= 3:
            frame_time_for_pose = 0
            pose_array.append(pose_array_temp)


        
    except:
        print("can't find faces")
        

    cv2.imshow('frame', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    temp_time = time.time() - frame_time
    no_of_frames += 1
    if temp_time >= 1:
        print("frames processed : ",no_of_frames)
        no_of_frames = 0
        frame_time = time.time()

cv2.waitKey(0)
vid.release()
cv2.destroyAllWindows()

In [2]:
img = cv2.imread("face_crowd1.jpg")
for face in x:
    x = face['facial_area'][0]
    y = face['facial_area'][1]
    h = face['facial_area'][2]# + face['facial_area']['h']
    w = face['facial_area'][3]# + face['facial_area']['w']

    frame = cv2.rectangle(frame,(x,y),(h,w),(255, 0, 0),thickness = 2)
cv2.imshow("output",img)

NameError: name 'x' is not defined

In [ ]:
import cv2
import mediapipe as mp

# initialize Pose estimator
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

pose = mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)


cap = cv2.VideoCapture('people-walking.mp4')

while cap.isOpened():
    # read frame from capture object
    _, frame = cap.read()

    try:
    	# convert the frame to RGB format
        RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(RGB)
        # process the RGB frame to get the result
        #print(results.pose_landmarks)

    # draw detected skeleton on the frame
        mp_drawing.draw_landmarks(
    	frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

    # show the final output
        cv2.imshow('Output', frame)
    except:
    	break

    if cv2.waitKey(1) == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

pose = mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5)

vid = cv2.VideoCapture(1)

while vid.isOpened():
    _, frame = vid.read()
    if 1:#try:
        results = pose.process(frame)
        print(results.pose_landmarks)
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow('output',frame)
    else:#except:
        break
    if cv2.waitKey(1) == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()


((array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ...

In [ ]:
video = cv2.VideoCapture(r'D:\\mini project 2\\temp_video.mp4')
  
# Used as counter variable
count = 0

# checks whether frames were extracted
success = 1

while success:
#for i in range(10):

    success, image = video.read()
    print(success)
    start_time = time.time()
    data = DeepFace.extract_faces("face_crowd2.png")
    print(time.time()-start_time)
    #print(data[0]['facial_area'])

    #if success:
        #cv2.imwrite("D:\\mini project 2\\vid to frame\\frame%d.jpg" % count, image)

    count += 1

In [1]:
from retinaface import RetinaFace
import cv2
frame = cv2.imread("face_crowd1.jpg")
facial_data = RetinaFace.detect_faces("face_crowd1.jpg")
for i in facial_data:
    print(i)

((array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ...

In [2]:
for face in facial_data:
    print(facial_data[face]['facial_area'])
    x = facial_data[face]['facial_area'][0]
    y = facial_data[face]['facial_area'][1]
    h = facial_data[face]['facial_area'][2] 
    w = facial_data[face]['facial_area'][3] 

    frame = cv2.rectangle(frame,(x,y),(h,w),(255, 0, 0),thickness = 2)
    cropped_image = frame[y:w,x:h] 
    #cv2.imshow("output",cropped_image) 
    cv2.imshow("frame1",frame)
cv2.waitKey()
cv2.destroyAllWindows()

[152, 20, 210, 94]
[1058, 410, 1152, 522]
[217, 257, 304, 362]
[970, 669, 1069, 789]
[701, 4, 761, 78]
[8, 441, 94, 557]
[828, 422, 910, 525]
[546, 0, 599, 70]
[540, 416, 628, 514]
[349, 441, 427, 536]
[1135, 9, 1195, 81]
[974, 243, 1052, 354]
[1370, 469, 1444, 577]
[486, 258, 559, 352]
[33, 84, 102, 182]
[941, 123, 1000, 203]
[1384, 43, 1440, 110]
[1364, 659, 1444, 780]
[340, 54, 400, 122]
[770, 99, 837, 191]
[1207, 270, 1276, 368]
[732, 321, 807, 410]
[524, 114, 597, 211]
[1356, 136, 1420, 217]
[1129, 103, 1193, 185]
[236, 101, 297, 181]
[636, 760, 746, 858]
[917, 0, 975, 63]
[322, 734, 419, 837]
[1384, 296, 1444, 385]


KeyboardInterrupt: 

In [1]:
from deepface import DeepFace
from retinaface import RetinaFace
import cv2
import time
import mediapipe as mp
import winsound
import numpy as np
import copy


def extract_persons(image):

    def get_output_layers(net):
        
        layer_names = net.getLayerNames()
        try:
            output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
        except:
            output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

        return output_layers


    def draw_prediction(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

        label = str(classes[class_id])

        color = COLORS[class_id]

        cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

        cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        

    Width = image.shape[1]
    Height = image.shape[0]
    scale = 0.00392

    classes = None

    with open("yolov3.txt", 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

    net = cv2.dnn.readNet("yolov3-tiny.weights", "yolo_tiny.cfg")

    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

    net.setInput(blob)

    outs = net.forward(get_output_layers(net))

    class_ids = []
    confidences = []
    boxes = []
    conf_threshold = 0.5
    nms_threshold = 0.4
    cropped_image_arr = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])


    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    for i in indices:
        try:
            box = boxes[i]
        except:
            i = i[0]
            box = boxes[i]
        
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        draw_prediction(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))
        try:
            cropped_image = image[round(y):round(y+h),round(x):round(x+w)] 
            cropped_image_arr.append(cropped_image) 
        except:
            pass
       

    cv2.imshow("object detection", image)
    cv2.waitKey()
        
    cv2.imwrite("object-detection.jpg", image)
    cv2.destroyAllWindows()
    return cropped_image_arr
vid = cv2.VideoCapture(1)
while 1:
    #ret, frame = vid.read()
    ret,frame = vid.read()
    extract_persons(frame)
    cv2.imshow("frame",frame)
    

((array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ...